In [ ]:
import polars as pl
from datetime import datetime
import get_and_store as gas
import polars_indicators as pi

In [ ]:
symbols = ['A','AA']

In [ ]:
df = gas.get_symbols_df(symbols)
df = df.filter(pl.col('Date') >= datetime(2023, 1, 1))
df = df.collect()

In [ ]:
df

In [ ]:
index_name = 'my_index'
column_name = "test"
mydf = df.with_row_count(name=index_name).with_columns( 
                pl.when(
                    pl.col(index_name) == 3) \
                    .then(pl.lit(True)) \
                .when(
                    (pl.col(index_name) == 4)) \
                    .then(pl.lit(True)) \
                .when(
                    (pl.col(index_name) == 7)) \
                    .then(pl.lit(True)) \
                .when(
                    (pl.col(index_name) == 85)) \
                    .then(pl.lit(True)) \
                .otherwise(
                    pl.lit(False)) \
                .alias(column_name))
mydf

In [ ]:
trail = pi.trailing_stop(mydf, 2)

In [ ]:
pi.create_trade_ids(trail.df, column_name, trail.column).df

In [ ]:
newcol = 'sum'
new = trail.df.with_columns(pl.col(trail.column).cumsum().shift(1).alias(newcol))
new
#left off here. issue with row index 84 being considered in the trade

In [ ]:
new = new.with_columns(pl.when(pl.col(column_name).max().over(pl.col(newcol))).then(pl.col('sum')).alias('test2'))
new

In [ ]:
new = new.with_columns(pl.col(column_name).max().over(pl.col(newcol)).alias('test3'))
new

In [ ]:
new.with_columns(pl.when(pl.col('Date') == pl.col('Date').min().over(pl.col('test2'),pl.col("Symbol"))).then(pl.lit(True)).otherwise(pl.lit(False)).cumsum().alias('test4'))

In [ ]:
new.with_columns(pl.when(pl.col(column_name)).then(-1).otherwise(pl.col(trail.column).cumsum()).alias('a'))

In [ ]:
pi.create_trade_ids(new, 'test', trail.column).df

In [ ]:
new = df

In [ ]:
index_name = 'index'
entries = "entries"
mydf = df.with_row_count(name=index_name).with_columns( 
                pl.when(
                    pl.col(index_name) == 3) \
                    .then(pl.lit(True)) \
                .when(
                    (pl.col(index_name) == 4)) \
                    .then(pl.lit(True)) \
                .when(
                    (pl.col(index_name) == 7)) \
                    .then(pl.lit(True)) \
                .when(
                    (pl.col(index_name) == 85)) \
                    .then(pl.lit(True)) \
                .otherwise(
                    pl.lit(False)) \
                .alias(entries))
mydf.head()

In [ ]:
trail = pi.trailing_stop(mydf, bars=2)
mydf = trail.df
mydf.head()

In [ ]:
exit_column = trail.column
exit_ids = "exit_ids"
enter_column = entries
df = mydf.with_columns(pl.col(exit_column).cumsum().shift(1).alias(exit_ids))
df.head()

In [ ]:
df.with_columns(
        pl.col(enter_column).max().over(pl.col(exit_ids)).alias("FileNotFoundError")) #ensures we only count trades with entries

In [ ]:


def add_id_column(df, enter_column, exit_column):
    def add_exit_ids(df, exit_column):
        column_name = "exit_ids_test"
        if column_name in df.columns:
            return pi.IndicatorResult(df, column_name)
        df = df.with_columns(pl.col(exit_column).cumsum().shift(1).alias(column_name))
        return pi.IndicatorResult(df, column_name)

    def add_traded_column(df, enter_column, exit_ids):
        column_name = "traded"
        if column_name in df.columns:
            return pi.IndicatorResult(df, column_name)
        df = df.with_columns(
            pl.col(enter_column).max().over(pl.col(exit_ids)).alias(column_name)) #ensures we only count trades with entries
        return pi.IndicatorResult(df, column_name)
    column_name = "ids"
    exit_ids = add_exit_ids(df, exit_column)
    traded = add_traded_column(exit_ids.df, enter_column, exit_ids.column)

    df = traded.df.with_columns(
        pl.when(
            pl.col(traded.column) #pl.col(enter_column).max().over(pl.col(exit_ids)) #ensures we only count trades with entries
            &
            (pl.col("Date") == pl.col("Date").min().over(exit_ids.column)) #ensures we only count a trade once per exit
        ).then(1).otherwise(0).alias(column_name))
    
    df = df.with_columns(pl.when(pl.col(traded.column)).then(pl.col(column_name).cumsum()))
    return pi.IndicatorResult(df, column_name)
ex = add_exit_ids(df, exit_column)
newdf = add_traded_column(ex.df, enter_column, ex.column).df
add_id_column(newdf, enter_column, exit_column).df


In [ ]:
lf = df.lazy()

In [ ]:
lf = lf.lazy()